# Notebook

Tempat saya membuat model

### Read dataset_anime_romance.csv in database

Pada part ini, saya coba baca file data anime.csv menggunakan pandas dan saya coba tampilkan 10 data teratas dan 10 data terbawah


In [5]:
# baca data dari file CSV

import pandas as pd

df = pd.read_csv('database/dataset_anime_romance.csv')

tabel_rapi = pd.concat([df.head(), df.tail()])

tabel_rapi

,mal_id,title,score,genres,synopsis,image_url,type,episodes,members,url
0,11757,Sword Art Online,7.22,"Action, Adventure, Fantasy, Romance",Ever since the release of the innovative Nerve...,https://cdn.myanimelist.net/images/anime/11/39...,TV,25.0,3262676,https://myanimelist.net/anime/11757/Sword_Art_...
1,23273,Shigatsu wa Kimi no Uso,8.64,"Drama, Romance","Kousei Arima is a child prodigy known as the ""...",https://cdn.myanimelist.net/images/anime/1405/...,TV,22.0,2386394,https://myanimelist.net/anime/23273/Shigatsu_w...
2,4224,Toradora!,8.04,"Drama, Romance",Ryuuji Takasu is a gentle high school student ...,https://cdn.myanimelist.net/images/anime/13/22...,TV,25.0,2339344,https://myanimelist.net/anime/4224/Toradora
3,21881,Sword Art Online II,6.72,"Action, Adventure, Fantasy, Romance","A year after escaping Sword Art Online, Kazuto...",https://cdn.myanimelist.net/images/anime/1223/...,TV,24.0,2090814,https://myanimelist.net/anime/21881/Sword_Art_...
4,37450,Seishun Buta Yarou wa Bunny Girl Senpai no Yum...,8.23,"Drama, Romance, Supernatural",The rare and inexplicable Puberty Syndrome is ...,https://cdn.myanimelist.net/images/anime/1301/...,TV,13.0,1924158,https://myanimelist.net/anime/37450/Seishun_Bu...
2252,61003,Seesaw,NaN,Romance,Music video for the song Seesaw by tuki..,https://cdn.myanimelist.net/images/anime/1773/...,Music,1.0,69,https://myanimelist.net/anime/61003/Seesaw
2253,62952,Xian Jian Qi Xia Zhuan Si,NaN,"Action, Drama, Fantasy, Romance","For the sake of their cultivation endeavors, t...",https://cdn.myanimelist.net/images/anime/1440/...,ONA,NaN,66,https://myanimelist.net/anime/62952/Xian_Jian_...
2254,62402,Dongzhi,NaN,"Drama, Romance","Shortly before he died, he had dated his old l...",https://cdn.myanimelist.net/images/anime/1177/...,Movie,1.0,59,https://myanimelist.net/anime/62402/Dongzhi
2255,63308,Tasogare ni Encore,NaN,"Drama, Romance",Music video for the song Tasogare ni Encore by...,https://cdn.myanimelist.net/images/anime/1022/...,Music,1.0,28,https://myanimelist.net/anime/63308/Tasogare_n...
2256,63313,Zui Hou Jue Ding Ai Shang Ni,NaN,"Comedy, Romance","Two strangers, both proposed marriage to their...",https://cdn.myanimelist.net/images/anime/1404/...,TV,13.0,0,https://myanimelist.net/anime/63313/Zui_Hou_Ju...


### Analisis Awal Struktur Data
<div style="text-align: left">
Berdasarkan tinjauan terhadap sampel data teratas (head) dan terbawah (tail), teridentifikasi beberapa karakteristik dataset yang memerlukan penanganan khusus sebelum pemrosesan lebih lanjut:

1. Kelengkapan Atribut: Data pada peringkat atas (populer) memiliki atribut yang lengkap. Namun, pada data peringkat bawah, ditemukan nilai kosong (missing values/NaN) pada kolom krusial synopsis dan score. Ketiadaan sinopsis pada entri tertentu (contoh: Di Ling Ai, Fukusuke) akan menghambat proses ekstraksi fitur berbasis teks.
2. Variasi Tipe Konten: Kolom type menunjukkan inkonsistensi format tontonan. Selain format naratif standar seperti 'TV' dan 'Movie', terdapat entri bertipe 'Music' (contoh: Into Me, Baby). Entri ini umumnya berupa video musik berdurasi pendek yang tidak relevan untuk sistem rekomendasi serial anime.
3. Distribusi Skor: Terdapat disparitas nilai pada kolom score, di mana entri baru atau kurang populer belum memiliki nilai rating (NaN), yang perlu diantisipasi saat melakukan pengurutan rekomendasi.
<.div>

### Kesimpulan Awal
Diperlukan tahap pembersihan data (data cleaning) untuk mengeliminasi baris dengan sinopsis kosong dan memfilter tipe konten non-naratif agar relevansi model tetap terjaga.

In [6]:
# pembersihan data

# NaN pada kolom synopsis dan filter tipe anime
df_clean = df.dropna(subset=['synopsis'])

# Buang data dengan tipe selain TV, Movie, OVA, ONA, Special
df_clean = df_clean[df_clean['type'].isin(['TV', 'Movie', 'OVA', 'ONA', 'Special'])]

df_clean = df_clean.reset_index(drop=True)

print("Hasil pembersihan data:")
print(f"Jumlah data awal  : {len(df)}")
print(f"Jumlah data akhir : {len(df_clean)}")
print(f"Data yang dibuang : {len(df) - len(df_clean)}")

Hasil pembersihan data:
Jumlah data awal  : 2257
Jumlah data akhir : 1972
Data yang dibuang : 285


### Analisis Hasil
<div style="text-align: left">
Proses pembersihan data berhasil menghapus 285 baris bermasalah, sehingga dataset kita sekarang tersisa 1.972 data yang valid. Ternyata, sekitar 12% dari data awal adalah "noise" berupa sinopsis kosong atau video musik yang tidak relevan. Langkah ini sangat penting untuk memastikan algoritma nanti berjalan lancar tanpa error, dan hasil rekomendasinya benar-benar berasal dari data cerita yang lengkap, bukan data yang rusak.
</div>

In [ ]:
# Vektorisasi TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(df_clean['synopsis'])

print("Ukuran Matrix:", tfidf_matrix.shape)

Ukuran Matrix: (1972, 16841)


### Analisis Hasil
Proses vektorisasi TF-IDF sudah berhasil mengubah seluruh sinopsis menjadi format angka dengan dimensi (1972, 16841). Angka ini menunjukkan bahwa dari 1.972 judul anime, sistem berhasil mengenali 16.841 kata kunci unik. Jumlah kosakata ini tergolong kaya, yang berarti dataset kita punya cukup banyak variasi kata untuk membedakan cerita satu sama lain secara detail. Ini modal penting agar hasil rekomendasinya nanti akurat dan tidak hanya berdasarkan kemiripan genre semata.

In [ ]:
# Hitung kemiripan kosinus (cosine similarity)

from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

print("Ukuran Matrix Similarity:", cosine_sim.shape)

Ukuran Matrix Similarity: (1972, 1972)


### Analisis Hasil

Perhitungan Cosine Similarity telah sukses menghasilkan matriks persegi berukuran (1972, 1972). Angka ini menunjukkan bahwa sistem sudah membandingkan "satu lawan satu" setiap judul anime dengan seluruh judul lainnya tanpa terkecuali. Jadi, setiap anime sekarang memiliki skor kemiripan spesifik terhadap 1.971 anime lain dalam database. Matriks inilah yang menjadi "otak" sistem rekomendasi kita; nantinya, sistem cukup mencari skor tertinggi dalam matriks ini untuk menentukan anime mana yang ceritanya paling mirip dengan pilihan pengguna.

In [10]:
# Membuat indeks berdasarkan judul anime

indices = pd.Series(df_clean.index, index=df_clean['title']).drop_duplicates()

print("Sword Art Online ada di baris ke:", indices['Sword Art Online'])

Sword Art Online ada di baris ke: 0


### Analisis Hasil
Mekanisme pemetaan (mapping) ini telah berhasil dibangun dan berfungsi sebagai jembatan antara input pengguna (judul anime) dan logika sistem (nomor baris). Hasil pengujian pada Sword Art Online yang mengembalikan nilai 0 memverifikasi bahwa sistem pencarian indeks bekerja akurat sesuai urutan data di tabel utama. Dengan adanya kamus indeks ini, kita dapat menerjemahkan judul anime apa pun menjadi koordinat angka secara instan, yang merupakan syarat mutlak untuk mengambil data skor kemiripan dari matriks yang sudah dibuat sebelumnya.

In [ ]:
# Fungsi rekomendasi anime

def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    anime_indices = [i[0] for i in sim_scores]

    return df_clean[['title', 'genres', 'score']].iloc[anime_indices]

# uji coba fungsi rekomendasi
get_recommendations('Sword Art Online')

,title,genres,score
3,Sword Art Online II,"Action, Adventure, Fantasy, Romance",6.72
526,Sword Art Online: Progressive Movie - Kuraki Y...,"Action, Adventure, Fantasy, Romance",7.69
1687,"Yume kara, Samenai",Romance,4.55
74,Accel World,"Action, Romance, Sci-Fi",7.20
138,Otome Game no Hametsu Flag shika Nai Akuyaku R...,"Comedy, Fantasy, Romance",7.44
170,Jaku-Chara Tomozaki-kun,"Drama, Romance",7.10
1019,Let's Play: Quest-darake no My Life,"Drama, Romance",6.75
1712,Nanaka 6/17 Special,"Comedy, Drama, Romance",6.54
1127,Chou Yuu Sekai: Being the Reality,"Action, Adventure, Fantasy, Romance",6.12
109,Netoge no Yome wa Onnanoko ja Nai to Omotta?,"Comedy, Romance, Ecchi",6.66


In [ ]:
# Fungsi untuk mencari judul anime berdasarkan kata kunci

def cari_anime(keyword):
    matches = df_clean[df_clean['title'].str.contains(keyword, case=False, regex=False)]
    
    return matches[['title', 'score']]

print("Hasil Pencarian 'sword'")
print(cari_anime('sword').head(5))
print("\n")

print("Hasil Pencarian 'otonari'")
print(cari_anime('otonari').head(5))

--- Hasil Pencarian 'titan' ---
                                                 title  score
0                                     Sword Art Online   7.22
3                                  Sword Art Online II   6.72
526  Sword Art Online: Progressive Movie - Kuraki Y...   7.69


--- Hasil Pencarian 'evangelion' ---
                                                 title  score
180  Otonari no Tenshi-sama ni Itsunomanika Dame Ni...   7.85
438                                   Otonari ni Ginga   7.16
711  Otonari no Tenshi-sama ni Itsunomanika Dame Ni...    NaN


In [ ]:
print("Hasil Pencarian 'Sword'")
print(cari_anime('Sword').head(5))
print("\n")

print("Hasil Pencarian 'Love'")
print(cari_anime('Love').head(5))

--- Hasil Pencarian 'Sword' ---
                                                 title  score
0                                     Sword Art Online   7.22
3                                  Sword Art Online II   6.72
526  Sword Art Online: Progressive Movie - Kuraki Y...   7.69


--- Hasil Pencarian 'Love' ---
                                                title  score
9   Yahari Ore no Seishun Love Comedy wa Machigatt...   8.00
23  Yahari Ore no Seishun Love Comedy wa Machigatt...   8.20
63  Yahari Ore no Seishun Love Comedy wa Machigatt...   8.35
88                                         To LOVE-Ru   6.96
98                                     Lovely★Complex   8.03
